# Preparation

In [1]:
# Install Ultralytics database
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6170.0/8062.4 GB disk)


## 1st step: Prediction

In [2]:
# Run inference on an image using YOLO11
!yolo predict model=yolo11n.pt source='https://ultralytics.com/images/zidane.jpg'

100%|███████████████████████████████████████| 5.35M/5.35M [00:00<00:00, 103MB/s]
Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

100%|██████████████████████████████████████| 49.2k/49.2k [00:00<00:00, 7.94MB/s]
image 1/1 /kaggle/working/zidane.jpg: 384x640 2 persons, 1 tie, 87.6ms
Speed: 10.3ms preprocess, 87.6ms inference, 268.5ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


## 2nd step: Validation

In [3]:
# Download COCO validation database
import torch
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')
!unzip -q tmp.zip -d datasets && rm tmp.zip

100%|██████████| 780M/780M [00:02<00:00, 309MB/s] 


In [4]:
# Validate YOLO11 on COCO8
!yolo val model=yolo11n.pt data=coco8.yaml

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

Dataset 'coco8.yaml' images not found ⚠️, missing path '/kaggle/working/datasets/coco8/images/val'
100%|████████████████████████████████████████| 433k/433k [00:00<00:00, 19.9MB/s]
Unzipping /kaggle/working/datasets/coco8.zip to /kaggle/working/datasets/coco8..
Dataset download success ✅ (0.7s), saved to /kaggle/working/datasets

100%|████████████████████████████████████████| 755k/755k [00:00<00:00, 26.0MB/s]
val: Scanning /kaggle/working/datasets/coco8/labels/val... 4 images, 0 backgroun
val: New cache created: /kaggle/working/datasets/coco8/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all          4         17       0.57       0.85      0.847      0.632
                person          3         10      0.557        0.6      0.585      0.27

## 3rd step: Train

In [5]:
logger = 'Comet'

if logger == 'Comet':
    %pip install -q comet_ml
    import comet_ml; comet_ml.login()
elif logger == 'TensorBoard':
    %load_ext tensorboard
    %tensorboard --logdir .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.8/725.8 kB 16.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 9.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)


Comet API key:  ········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [6]:
# YOLO11 on COCO8 training (max. 3 epochs)
!yolo train model=yolo11n.pt data=coco8.yaml epochs=3 imgsz=640

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=coco8.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True

## 4th step: Export

In [7]:
!yolo export model=yolo11n.pt format=torchscript

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.00GHz)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs

PyTorch: starting from 'yolo11n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (5.4 MB)

TorchScript: starting export with torch 2.5.1+cu121...
TorchScript: export success ✅ 2.0s, saved as 'yolo11n.torchscript' (10.5 MB)

Export complete (3.8s)
Results saved to /kaggle/working
Predict:         yolo predict task=detect model=yolo11n.torchscript imgsz=640  
Validate:        yolo val task=detect model=yolo11n.torchscript imgsz=640 data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app
💡 Learn more at https://docs.ultralytics.com/modes/export


# Python Usage

In [8]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolo11n.yaml')
model = YOLO('yolo11n.pt')

# Use the model
results = model.train(data='coco8.yaml', epochs=3)
results = model.val()
results = model('https://ultralytics.com/images/bus.jpg')
results = model.export(format='onnx')

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=coco8.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jamessuryaputra/general/ad32eecff9294ee680ba1766c6e06a47



TensorBoard: Start with 'tensorboard --logdir runs/detect/train2', view at http://localhost:6006/


COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


...

/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.3ms preprocess, 16.7ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train22



100%|██████████| 134k/134k [00:00<00:00, 11.1MB/s]

image 1/1 /kaggle/working/bus.jpg: 640x480 4 persons, 1 bus, 41.5ms
Speed: 2.9ms preprocess, 41.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)


Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (5.3 MB)
requirements: Ultralytics requirements ['onnxslim', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.9/142.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 248.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 225.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 317.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 10.2s, installed 2 packages: ['onnxslim', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.48...
ONNX: export success ✅ 11.5s, saved as 'runs/detect/train2/weights/best.onnx' (10.2

# YOLO11 Tasks

## 1st task: Detection

In [9]:
# Load the model (yolo11n.pt) and train on COCO128 (for 3 epochs), then predict the image
from ultralytics import YOLO

model = YOLO('yolo11n.pt')
model.train(data='coco8.yaml', epochs=3)
model('https://ultralytics.com/images/bus.jpg')

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=coco8.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tru

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jamessuryaputra/general/565a2c8741424b8dabdd2d1bb41f52d6

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


TensorBoard: Start with 'tensorboard --logdir runs/detect/train3', view at http://localhost:6006/
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


train: Scanning /kaggle/working/datasets/coco8/labels/train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/train3


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : cooperative_burbot_2076
COMET INFO:     url                   : https://www.comet.com/jamessuryaputra/general/565a2c8741424b8dabdd2d1bb41f52d6
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [7]               : (0.0, 8.092e-07)
COMET INFO:     lr/pg1 [7]               : (0.0, 8.092e-07)
COMET INFO:     lr/pg2 [7]               : (0.0, 8.092e-07)
COMET INFO:     metrics/mAP50(B) [8]     : (0.84889, 0.85565)
COMET INFO:     metrics/mAP50-95(B) [8]  : (0.61315, 0.6332009154276238)
COMET INFO:     metrics/precision(B) [8] : (0.57544, 0.58079)
COMET INFO:     metrics/recall(B)        : 0.85
COMET INFO:     model/GFLOPs             : 6.614


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /kaggle/working/bus.jpg: 640x480 4 persons, 1 bus, 9.7ms
Speed: 2.2ms preprocess, 9.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted p

## 2nd task: Segmentation

In [10]:
# Load the model (yolo11n-seg) and train on COCO128 (for 3 epochs), then predict the image
from ultralytics import YOLO

model = YOLO('yolo11n-seg.pt')
model.train(data='coco8-seg.yaml', epochs=3)
model('https://ultralytics.com/images/bus.jpg')

100%|██████████| 5.90M/5.90M [00:00<00:00, 118MB/s]

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=coco8-seg.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b


100%|██████████| 439k/439k [00:00<00:00, 22.7MB/s]
Unzipping /kaggle/working/datasets/coco8-seg.zip to /kaggle/working/datasets/coco8-seg...: 100%|██████████| 25/25 [00:00<00:00, 4045.43file/s]

Dataset download success ✅ (0.6s), saved to /kaggle/working/datasets


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7              

YOLO11n-seg summary: 203 layers, 2,876,848 parameters, 2,876,832 gradients, 10.5 GFLOPs

Transferred 561/561 items from pretrained weights


...

/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 5.6ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/segment/train


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : binding_egret_961
COMET INFO:     url                   : https://www.comet.com/jamessuryaputra/general/4152ced0a1624bd88cbe259c850f66a8
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [7]               : (0.0, 8.092e-07)
COMET INFO:     lr/pg1 [7]               : (0.0, 8.092e-07)
COMET INFO:     lr/pg2 [7]               : (0.0, 8.092e-07)
COMET INFO:     metrics/mAP50(B) [8]     : (0.93911, 0.94056)
COMET INFO:     metrics/mAP50(M) [8]     : (0.82163, 0.8222768981809462)
COMET INFO:     metrics/mAP50-95(B) [8]  : (0.66678, 0.66836)
COMET INFO:     metrics/mAP50-95(M) [8]  : (0.56376, 0.56657)
COMET INFO:     metrics/precision(B) [8]


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /kaggle/working/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 18.5ms
Speed: 2.3ms preprocess, 18.5ms inference, 10.2ms postprocess per image at shape (1, 3, 640, 480)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 

## 3rd task: Classification

In [11]:
# Load the model (yolo11n-cls) and train on COCO128 (for 3 epochs), then predict the image
from ultralytics import YOLO

model = YOLO('yolo11n-cls.pt')
model.train(data='mnist160', epochs=3)
model('https://ultralytics.com/images/bus.jpg')

100%|██████████| 5.52M/5.52M [00:00<00:00, 110MB/s]

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=mnist160, epochs=3, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=


100%|██████████| 70.0k/70.0k [00:00<00:00, 8.74MB/s]
Unzipping /kaggle/working/datasets/mnist160.zip to /kaggle/working/datasets/mnist160...: 100%|██████████| 184/184 [00:00<00:00, 10637.08file/s]

Dataset download success ✅ (0.7s), saved to /kaggle/working/datasets/mnist160

train: /kaggle/working/datasets/mnist160/train... found 80 images in 10 classes ✅ 
val: None...
test: /kaggle/working/datasets/mnist160/test... found 80 images in 10 classes ✅ 
Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                 


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jamessuryaputra/general/c6b5f7dc9e314794b5fce2c2621636ee

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


...

               classes   top1_acc   top5_acc: 100%|██████████| 3/3 [00:00<00:00, 27.67it/s]
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all     0.0875      0.538
Speed: 0.1ms preprocess, 0.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : rear_transom_3582
COMET INFO:     url                   : https://www.comet.com/jamessuryaputra/general/c6b5f7dc9e314794b5fce2c2621636ee
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [7]                : (2.856e-05, 4.3054200000000005e-05)
COMET INFO:     lr/pg1 [7]                : (2.856e-05, 4.3054200000000005e-05)
COMET INFO:     lr/pg2 [7]                : (2.856e-05, 4.3054200000000005e-05)
COMET INFO:     metrics/accuracy_top1 [8] : (0.08749999850988388, 0.1375)
COMET INFO:     metrics/accuracy_top5 [8] : (0.475, 0.5375000238418579)
COMET INFO:     model/GFLOPs              : 3.263
COMET INFO:     model/parameters          :


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /kaggle/working/bus.jpg: 224x224 6 0.35, 9 0.15, 5 0.14, 0 0.09, 8 0.08, 4.3ms
Speed: 13.4ms preprocess, 4.3ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9'}
 obb: None
 orig_img: array([[[119, 146, 172],
         [121, 148, 174],
         [122, 152, 177],
         ...,
         [161, 171, 188],
         [160, 170, 187],
         [160, 170, 187]],
 
        [[120, 147, 173],
         [122, 149, 175],
         [123, 153, 178],
         ...,
         [161, 171, 188],
         [160, 170, 187],
         [160, 170, 187]],
 
        [[123, 150, 176],
         [124, 151, 177],
         [125, 155, 180],
         ...,
         [161, 171, 188],
         [160, 170, 187],
         [160, 170, 187]],
 
        ...,
 
        [[183, 182, 186],
         [179, 178, 182],
         [180, 179, 183],
         ...,
         [121, 111, 117],
         [113, 103, 109],
         [115, 105, 111]],
 
        [[165, 164, 168],
         [173, 172, 176],
         [187, 186, 190],


## 4th step: Pose

In [12]:
# Load the model (yolo11n-pose) and train on COCO128 (for 3 epochs), then predict the image
from ultralytics import YOLO

model = YOLO('yolo11n-pose.pt')
model.train(data='coco8-pose.yaml', epochs=3)
model('https://ultralytics.com/images/bus.jpg')

100%|██████████| 5.97M/5.97M [00:00<00:00, 128MB/s]

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=pose, mode=train, model=yolo11n-pose.pt, data=coco8-pose.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo


100%|██████████| 334k/334k [00:00<00:00, 17.4MB/s]
Unzipping /kaggle/working/datasets/coco8-pose.zip to /kaggle/working/datasets/coco8-pose...: 100%|██████████| 27/27 [00:00<00:00, 4739.92file/s]

Dataset download success ✅ (0.6s), saved to /kaggle/working/datasets

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128,

YOLO11n-pose summary: 196 layers, 2,874,462 parameters, 2,874,446 gradients, 7.5 GFLOPs

Transferred 541/541 items from pretrained weights


...

/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 4.4ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/pose/train


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : large_spoonbill_4169
COMET INFO:     url                   : https://www.comet.com/jamessuryaputra/general/96bdb91983b24753b8f5b6b560b48443
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [7]               : (0.0, 1.36e-05)
COMET INFO:     lr/pg1 [7]               : (0.0, 1.36e-05)
COMET INFO:     lr/pg2 [7]               : (0.0, 1.36e-05)
COMET INFO:     metrics/mAP50(B) [8]     : (0.90686, 0.9125011180679786)
COMET INFO:     metrics/mAP50(P) [8]     : (0.72218, 0.76275)
COMET INFO:     metrics/mAP50-95(B) [8]  : (0.69744, 0.7119635713178295)
COMET INFO:     metrics/mAP50-95(P) [8]  : (0.35574, 0.36294)
COMET INFO:     metrics/preci


Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /kaggle/working/bus.jpg: 640x480 4 persons, 21.1ms
Speed: 3.3ms preprocess, 21.1ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: ultralytics.engine.results.Keypoints object
 masks: None
 names: {0: 'person'}
 obb: None
 orig_img: array([[[119, 146, 172],
         [121, 148, 174],
         [122, 152, 177],
         ...,
         [161, 171, 188],
         [160, 170, 187],
         [160, 170, 187]],
 
        [[120, 147, 173],
         [122, 149, 175],
         [123, 153, 178],
         ...,
         [161, 171, 188],
         [160, 170, 187],
         [160, 170, 187]],
 
        [[123, 150, 176],
         [124, 151, 177],
         [125, 155, 180],
         ...,
         [161, 171, 188],
         [160, 170, 187],
         [160, 170, 187]],
 
        ...,
 
        [[183, 182, 186],
         [179, 178, 182],
         [180, 179, 183],
         ...,
         [121, 111, 117],
         [113, 103, 109],
         [115, 105, 111]],
 
        [[165, 164, 168],
         [173, 172, 176],
         [187, 186,

## 5th task: Oriented Bounding Boxes (OBB)

In [13]:
# Load the model (yolo11n-obb) and train on COCO128 (for 3 epochs), then predict the image
from ultralytics import YOLO

model = YOLO('yolo11n-obb.pt')
model.train(data='dota8.yaml', epochs=3)
model('https://ultralytics.com/images/boats.jpg')

100%|██████████| 5.53M/5.53M [00:00<00:00, 112MB/s]

Ultralytics 8.3.89 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=obb, mode=train, model=yolo11n-obb.pt, data=dota8.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tr


100%|██████████| 1.24M/1.24M [00:00<00:00, 36.4MB/s]
Unzipping /kaggle/working/datasets/dota8.zip to /kaggle/working/datasets/dota8...: 100%|██████████| 27/27 [00:00<00:00, 1979.93file/s]

Dataset download success ✅ (0.7s), saved to /kaggle/working/datasets

Overriding model.yaml nc=80 with nc=15

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128

YOLO11n-obb summary: 196 layers, 2,664,432 parameters, 2,664,416 gradients, 6.7 GFLOPs

Transferred 541/541 items from pretrained weights


...

Speed: 0.4ms preprocess, 6.8ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs/obb/train


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : productive_damper_8035
COMET INFO:     url                   : https://www.comet.com/jamessuryaputra/general/708ddc990a6c44e6879e86af4693739e
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [7]               : (0.0, 3.5768e-06)
COMET INFO:     lr/pg1 [7]               : (0.0, 3.5768e-06)
COMET INFO:     lr/pg2 [7]               : (0.0, 3.5768e-06)
COMET INFO:     metrics/mAP50(B)         : 0.995
COMET INFO:     metrics/mAP50-95(B) [8]  : (0.80425, 0.8043716666666667)
COMET INFO:     metrics/precision(B) [8] : (0.94439, 0.9459)
COMET INFO:     metrics/recall(B)        : 1.0
COMET INFO:     model/GFLOPs             : 6.703
COMET INFO: 

100%|██████████| 190k/190k [00:00<00:00, 11.8MB/s]

image 1/1 /kaggle/working/boats.jpg: 576x1024 51.2ms
Speed: 4.1ms preprocess, 51.2ms inference, 5.2ms postprocess per image at shape (1, 3, 576, 1024)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: 'plane', 1: 'ship', 2: 'storage tank', 3: 'baseball diamond', 4: 'tennis court', 5: 'basketball court', 6: 'ground track field', 7: 'harbor', 8: 'bridge', 9: 'large vehicle', 10: 'small vehicle', 11: 'helicopter', 12: 'roundabout', 13: 'soccer ball field', 14: 'swimming pool'}
 obb: ultralytics.engine.results.OBB object
 orig_img: array([[[ 40,  68, 103],
         [ 29,  56,  90],
         [ 14,  36,  64],
         ...,
         [ 86, 100,  94],
         [ 88, 102,  96],
         [ 92, 106, 100]],
 
        [[ 64,  95, 128],
         [ 54,  83, 114],
         [ 39,  66,  92],
         ...,
         [ 83,  97,  91],
         [ 82,  96,  90],
         [ 82,  96,  90]],
 
        [[ 77, 117, 142],
         [ 73, 111, 135],
         [ 69, 103, 126],
         ...,
         [ 62,  74,  68],
         [ 64,  78,  72],
         [ 65,  79,  73]],
 
        ...,
 
        [[ 62,  6